<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Merging and Concatenation with pandas




### Learning Objectives
- Practice concatenating dataframes using pandas
- Join pandas dataframes using SQL-style join operations
- Remove duplicate rows or columns

### Lesson Guide
- [Overview of concatenation and joining](#introduction)
- [Concatenation using pandas](#pandas_concatenation)
- [Identifying and removing duplicates](#removing_duplicates)
- [SQL-style joins using pandas](#pandas_joins)

<a id='introduction'></a>

### Overview of concatenation and joining

---

**Concatenation** is the process of joining separate objects along a dimension to create a new single object. In
computer programming and data processing, two or more character strings are sometimes concatenated for the purpose of saving space or so that they can be addressed as a **single item**.

In pandas, we will be concatenating dataframes together **along rows or columns**. 

**Joins** with pandas happen when columns of two DataFrames are **joined together on index or on a key column**. The concept is the same as **SQL joins**. In pandas, joins are done typically with the `.merge()` function.


<a id='pandas_concatenation'></a>

### Concatenation using pandas

---

It is often the case that you  would like to concatenate two dataframes together. Perhaps your data is split up into two groups of subjects with the same variables/columns and you want to join them together (stacking vertically - adding rows). Or perhaps you have new variables for all of your existing subjects (stacking horizontally - adding columns).

Below we have two simple datasets we can use to practice pandas concatenation.

In [1]:
import pandas as pd

In [53]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[1, 3, 6, 7])

In [4]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In pandas we can use the `pd.concat` function to stack DataFrames vertically or horizontally. `pd.concat()` takes a list of pandas dataframes as its first argument, and then an axis keyword argument indicating how to concatenate the dataframes. 

Setting `axis=0` will concatenate the DataFrames vertically (adding rows)

**Concatenate `df1` and `df2` by stacking them vertically.**

In [54]:
df2

,A,B,C,D
1,A4,B4,C4,D4
3,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [8]:
# Vertical concatenation
pd.concat([df1, df2], axis=0)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


**Concatenate `df1` and `df2` by stacking them horizontally.**

In [55]:
# Horizontal concatenation
pd.concat([df1, df2], axis=1)

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,A4,B4,C4,D4
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,A5,B5,C5,D5
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7


In [17]:
df2.reset_index(drop=True)

,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


You can see that because the pandas **index** are different for the two dataframes, it fills in null values. Perhaps we don't care about the row labels during the horizontal concatenation. If you reset the index for `df2` prior to the concatenation it will not fill in null values:

In [52]:
# Horizontal concatenation ignoring row labels
pd.concat([df1, df2.reset_index(drop=True)], axis=1)


,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,A3,B3,C3,D3,A7,B7,C7,D7


,A,A,B,B
0,A0,A4,B0,B4
1,A1,A5,B1,B5
2,A2,A6,B2,B6
3,A3,A7,B3,B7


<a id='removing_duplicates'></a>

### Identifying and Removing Duplicates

Sometimes when we combine data we may come across data that is duplicated. For a data set that has millions of rows, it will be difficult to identify these duplicated rows manually. We should always check for duplicate data, especially when one of the columns must be unique.

Let's see how we can do this using a sample DataFrame


In [34]:
df3 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3', 'A1', 'A4','A1'],
                    'B': ['B0', 'B1', 'B2', 'B3', 'B1', 'B4','B1'],
                    'C': ['C0', 'C1', 'C2', 'C3', 'C1', 'C2','C1'],
                    'D': ['D0', 'D1', 'D2', 'D3', 'D1', 'D3','D1']})



In [35]:
# Are there duplicates? It's hard to tell...
df3

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A1,B1,C1,D1
5,A4,B4,C2,D3
6,A1,B1,C1,D1


### Check for duplicates

The `duplicated` function returns a Series with boolean values showing whether a row is a duplicate.

In [37]:
df3.duplicated(keep='last')


0    False
1     True
2    False
3    False
4     True
5    False
6    False
dtype: bool

Use the `subset=` parameter to check for duplicates on specific columns, for example a column that should be a unique value.

Use the `keep=` parameter to determine which row should be considered a duplicate.

In [38]:
# Default values: subset=None, keep='first'
df3.duplicated(subset=['C'], keep='last')


0    False
1     True
2     True
3    False
4     True
5    False
6    False
dtype: bool

In [40]:
# We can use the boolean mask to show the specific duplicated rows
df3[df3.duplicated(subset=['C'])]

,A,B,C,D
4,A1,B1,C1,D1
5,A4,B4,C2,D3
6,A1,B1,C1,D1


### Removing duplicate rows

Once you have determined the duplicate rows should be removed, you can use `drop_duplicates`. To change the original DataFrame you would have to set `inplace=True`

In [41]:
# Drop the duplicates
df3.drop_duplicates(subset=['C'], keep='first', inplace=False)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [43]:
# Set inplace=True and check that the duplicates are really removed
df3.drop_duplicates(subset=['C'], keep='first', inplace=True)

In [44]:
df3

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


<a id='pandas_joins'></a>

### Left, right, inner, and outer joins in pandas

---

The pandas `merge` function allows us to join together DataFrames using columns as keys.

Below we have two dataframes with information on `subject_id`, `first_name`, and `last_name`. We also have a third dataframe with information on `subject_id` and `test_id`.

In [45]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data)
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [46]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data)
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [47]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_n = pd.DataFrame(raw_data)
df_n

,subject_id,test_id
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


**Pandas `pd.merge()` for SQL-style joins**

A left join produces a complete set of records from `df_a`, with the matching records (where available) in `df_b`. If there is no match, the right side will contain null.

The pandas `pd.merge()` command has arguments:
- left-hand dataset
- right-hand dataset
- `on=` : keyword argument specifying the key column to join the dataframes on
- `how=` : keyword argument specifying the type of join (left, right, inner, outer)

#### Left join `df_b` onto `df_a` by `subject_id`.

In [48]:
# left join
pd.merge(df_a, df_b, on='subject_id', how='left')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


In [49]:
df_a.merge(df_b, on='subject_id', how='left')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black


In [50]:
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


#### Right join `df_b` onto `df_a` by `subject_id`

Merge with a right join produces a complete set of records from `df_b`, with the matching records (where available) in `df_a`. If there is no match, the left side will contain null.


In [56]:
# right join
pd.merge(df_a, df_b, on='subject_id', how='right')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black
2,6,NaN,NaN,Bran,Balwner
3,7,NaN,NaN,Bryce,Brice
4,8,NaN,NaN,Betty,Btisan


#### Outer join `df_b` onto `df_a` by `subject_id`

An outer join produces the set of all records in `df_a` and `df_b`, with matching records from both sides where available. If there is no match, the missing side will contain null.

In [59]:
# outer join
pd.merge(df_a, df_b, on='subject_id', how='outer')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,Alex,Anderson,NaN,NaN
1,2,Amy,Ackerman,NaN,NaN
2,3,Allen,Ali,NaN,NaN
3,4,Alice,Aoni,Billy,Bonder
4,5,Ayoung,Atiches,Brian,Black
5,6,NaN,NaN,Bran,Balwner
6,7,NaN,NaN,Bryce,Brice
7,8,NaN,NaN,Betty,Btisan


#### Inner join `df_b` onto `df_a` by `subject_id`

An inner join produces only the set of records that match in both df_a and df_b.

In [60]:
# inner join
pd.merge(df_a, df_b, on='subject_id', how='inner')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y
0,4,Alice,Aoni,Billy,Bonder
1,5,Ayoung,Atiches,Brian,Black


#### Combine the information in `df_a`, `df_b` and `df_n` using joins

No information should be lost.

In [63]:
# A:
merged_df_ab=pd.merge(df_a,df_b,on='subject_id',how='outer')
pd.merge(merged_df_ab, df_n, on='subject_id',how='outer')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y,test_id
0,1,Alex,Anderson,NaN,NaN,51.0
1,2,Amy,Ackerman,NaN,NaN,15.0
2,3,Allen,Ali,NaN,NaN,15.0
3,4,Alice,Aoni,Billy,Bonder,61.0
4,5,Ayoung,Atiches,Brian,Black,16.0
5,6,NaN,NaN,Bran,Balwner,NaN
6,7,NaN,NaN,Bryce,Brice,14.0
7,8,NaN,NaN,Betty,Btisan,15.0
8,9,NaN,NaN,NaN,NaN,1.0
9,10,NaN,NaN,NaN,NaN,61.0


In [64]:

pd.merge(pd.merge(df_a,df_b,on='subject_id',how='outer'), df_n, on='subject_id',how='outer')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y,test_id
0,1,Alex,Anderson,NaN,NaN,51.0
1,2,Amy,Ackerman,NaN,NaN,15.0
2,3,Allen,Ali,NaN,NaN,15.0
3,4,Alice,Aoni,Billy,Bonder,61.0
4,5,Ayoung,Atiches,Brian,Black,16.0
5,6,NaN,NaN,Bran,Balwner,NaN
6,7,NaN,NaN,Bryce,Brice,14.0
7,8,NaN,NaN,Betty,Btisan,15.0
8,9,NaN,NaN,NaN,NaN,1.0
9,10,NaN,NaN,NaN,NaN,61.0


In [65]:
pd.merge(df_n, pd.merge(df_a,df_b,on='subject_id',how='outer'),on='subject_id',how='outer')

,subject_id,test_id,first_name_x,last_name_x,first_name_y,last_name_y
0,1,51.0,Alex,Anderson,NaN,NaN
1,2,15.0,Amy,Ackerman,NaN,NaN
2,3,15.0,Allen,Ali,NaN,NaN
3,4,61.0,Alice,Aoni,Billy,Bonder
4,5,16.0,Ayoung,Atiches,Brian,Black
5,7,14.0,NaN,NaN,Bryce,Brice
6,8,15.0,NaN,NaN,Betty,Btisan
7,9,1.0,NaN,NaN,NaN,NaN
8,10,61.0,NaN,NaN,NaN,NaN
9,11,16.0,NaN,NaN,NaN,NaN


#### Combine the information in the three datasets only where information is contained in all rows of the output.

In [66]:
# A: 
pd.merge(pd.merge(df_a,df_b,on='subject_id',how='inner'), df_n, on='subject_id',how='inner')

,subject_id,first_name_x,last_name_x,first_name_y,last_name_y,test_id
0,4,Alice,Aoni,Billy,Bonder,61
1,5,Ayoung,Atiches,Brian,Black,16


In [ ]:
df_a.merge(df_b, on='subject_id', how='outer').merge(df_n, on='subject_id', how='outer')